# Hierarchical Clustering

## Autoreload

Autoreload allows the notebook to dynamically load code: if we update some helper functions *outside* of the notebook, we do not need to reload the notebook.

In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
import numpy as np
import pandas as pd
import os
import sys
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import procyclingstats as pcs
import umap

from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import AgglomerativeClustering

# Otherwise nothing will be found
sys.path.append(os.path.abspath(os.path.join('..')))

from utility.cluster_utility import scale_data, hier_search, get_average_per_cluster

/home/yuri/anaconda3/envs/hlt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-30 10:47:32.148015: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Global variables, constants, etc.

In [3]:
sys.path.append('../dataset/')
sys.path.append('../utility/')

# Global variables, constants
IMAGE_DIR = '../images/'
RANDOM_STATE = 42
hyperparametrs = {
    'n_clusters': np.arange(3, 10), # starting from 3 because with 2 clusters we just get outlier detection
    'linkage': ['single', 'complete', 'average', 'ward']
}
RUN_GRID_SEARCH = True

We've spent a lot of effort into engineering new features, understanding them, etc. Whether we've done this well and whether is enough it's a different story.

The point here is that - after the understanding, engineering and transformation - we have some cleaned-up versions of the datasets.

In [4]:
cyclists_df = pd.read_csv('../dataset/cyclists_cleaned.csv')
races_df = pd.read_csv('../dataset/races_cleaned.csv')

Let's reduce the dimensionality of the dataset by dropping useless or non-descriptive columns. The last three columns of `cyclists_df` have been dropped because of their correlation with other columns and the presence of strong outliers

In [5]:
cyclists_df = cyclists_df.select_dtypes(include=[np.number])
cyclists_df = cyclists_df.drop(columns=['birth_year', 'weight', 'height', 'avg_points_per_race', 'mean_stamina_index', 'victories_by_points'])
cyclists_df = cyclists_df.dropna()

In [6]:
races_df = races_df.select_dtypes(include=[np.number])
races_df = races_df.drop(columns=['uci_points', 'quality_adjusted_points', 'position', 'delta', 'cyclist_age', 'age_performance_index', 'profile', 
                                  'season', 'norm_points', 'normalized_length', 'normalized_quality', 'normalized_steepness', 'time_seconds', 'normalized_time', 'stamina_index'])
races_df = races_df.dropna()

Because of its high resemblance to a categorical variable (check the data understanding notebook for proof), we decided to remove the `points` column to weed out bad clusters. While `steepness` defines a relation between `length` and `climb_total`, we thought that hierarchical clustering could benefit from this feature because it is quite correlated with `profile`, which is categorical, and is a continuous variable with a distribution that has no sensational outliers (thus not impairing the clustering algorithms).

In [7]:
races_df = races_df.drop(columns=['points'])
races_df = races_df.drop_duplicates()

# Cyclists

Let's scale the data

In [8]:
cyc_normalizer, cyc_normalized = scale_data(cyclists_df)

Let's check how the silhouette scores for the selected linkage types and maximum clusters compare

In [9]:
if RUN_GRID_SEARCH: search_results = hier_search(hyperparametrs, cyc_normalized)

/home/yuri/anaconda3/envs/hlt/lib/python3.11/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 28 is smaller than n_iter=80. Running 28 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Let's check them

In [10]:
if RUN_GRID_SEARCH: display(search_results)

,n_clusters,linkage,random_state,silhouette
0,3,single,42,0.775487
1,4,single,42,0.729948
14,3,average,42,0.461807
2,5,single,42,0.449895
3,6,single,42,0.425771
4,7,single,42,0.423144
15,4,average,42,0.420770
5,8,single,42,0.419791
7,3,complete,42,0.393271
16,5,average,42,0.371129


Looks like the best silhouette score is reached by single linkages with 3 clusters. 

Let's try to plot it

In [12]:
single_linkage = AgglomerativeClustering(n_clusters=3, linkage='single')
labels = single_linkage.fit_predict(cyc_normalized)

In [13]:
cyclists_df['cluster'] = labels

In [14]:
# plot all pairs of values and color by cluster
sns.pairplot(cyclists_df, hue='cluster')
plt.show()

KeyboardInterrupt: 

And it just looks like a mediocre result for outlier detection! As expected, MIN suffers a lot from outliers and noise. Let's check how the best average works

In [ ]:
average_linkage = AgglomerativeClustering(n_clusters=3, linkage='average')
labels = average_linkage.fit_predict(cyc_normalized)

In [ ]:
cyclists_df['cluster'] = labels

In [ ]:
# plot all pairs of values and color by cluster
sns.pairplot(cyclists_df, hue='cluster')
plt.show()

It's still suffering, albeit doing a bit better. The main reason seems to be some cyclists having totalized a great deal of points. Let's check how MAX goes.

In [ ]:
# show only the rows of silhouettes where linkage = 'complete'

if RUN_GRID_SEARCH: search_results[search_results['linkage'] == 'complete']

In [ ]:
complete_linkage = AgglomerativeClustering(n_clusters=3, linkage='complete')
labels = complete_linkage.fit_predict(cyc_normalized)
cyclists_df['cluster'] = labels

In [ ]:
# plot all pairs of values and color by cluster
sns.pairplot(cyclists_df, hue='cluster')
plt.show()

Well, a bit of a more satisfying division.

In [ ]:
avg_cycl, _ = get_average_per_cluster(labels, cyclists_df)
avg_cycl

The first cluster represents experienced, older cyclists who participate in a good number of races and perform moderately well. <br>
The second cluster represents a small group of competitive cyclists that perform exceedingly well and have the highest experience. This is basically a set of outliers <br>
The third cluster consists of younger and less experienced cyclists that tend to end up in poorer positions.


Let's check how Ward's Method fared

In [ ]:
# show only the rows of silhouettes where linkage = 'complete'

if RUN_GRID_SEARCH: search_results[search_results['linkage'] == 'ward']

In [ ]:
ward_linkage = AgglomerativeClustering(n_clusters=5, linkage='ward')
labels = ward_linkage.fit_predict(cyc_normalized)
cyclists_df['cluster'] = labels

In [ ]:
# plot all pairs of values and color by cluster
sns.pairplot(cyclists_df, hue='cluster')
plt.show()

In [ ]:
avg_cycl, _ = get_average_per_cluster(labels, cyclists_df)
avg_cycl

Division seem to be mainly focused on the number of races and age. <br>
The first is the largest group of cyclists, on the young-middle age side, who partecipated in quite some races but tend to have mediocre performances. <br>
The second represents more experienced cyclists that accumulated a discrete number of points. Their performance is better, albeit still mediocre, and their average speed is slightly above other clusters <br>
The third consists of the youngest and most lightweight cyclists who perform almost as well as the previous cluster. Albeit inexperienced, they are the fastest cluster. <br>
The fourth seems to represent a group of the best, most experienced cyclists with great performance metrics. <br>
The last cluster represents, probably, historical cyclists that performed exceptionally well. It is interesting to note that their BMI is higher, compared to other groups, and that their race count isn't that high, probably due to not much data being cleanly collected on them


Let's see how this clustering appears in tridimensional space. We'll use UMAP to reduce its dimensionality.

In [ ]:
reducer = umap.UMAP(n_neighbors=15, learning_rate=0.1, n_components=3, random_state=RANDOM_STATE)

In [ ]:
reduced_data = reducer.fit_transform(cyc_normalized)

In [ ]:
# Prepare data for Plotly
df = pd.DataFrame({
    'UMAP 1': reduced_data[:, 0],
    'UMAP 2': reduced_data[:, 1],
    'UMAP 3': reduced_data[:, 2],
    'Cluster': ward_linkage.labels_
})

# Create interactive 3D scatter plot
fig = px.scatter_3d(
    df,
    x='UMAP 1',
    y='UMAP 2',
    z='UMAP 3',
    color='Cluster',
    title='UMAP 3D',
    color_continuous_scale='Viridis'
)

fig.show()

Looks like the clustering in three dimensions keeps some sort of homogenity, albeit the clusters appear to be all but globular (which is expected from hierarchical clustering).

# Races

Let's scale the data

In [9]:
rac_normalizer , rac_normalized = scale_data(races_df)

Let's check how the silhouette scores for the selected linkage types and maximum clusters compare

In [ ]:
if RUN_GRID_SEARCH: 
    search_results = hier_search(hyperparametrs, rac_normalized, samples = 80)
    display(search_results)

/home/yuri/anaconda3/envs/hlt/lib/python3.11/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 28 is smaller than n_iter=80. Running 28 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


MemoryError: Unable to allocate 11.0 GiB for an array with shape (1473294903,) and data type float64

Let's see how the first three results fared out

## First Linkage

In [ ]:
first_linkage_races = AgglomerativeClustering(n_clusters=3, linkage='average')
second_linkage_races = AgglomerativeClustering(n_clusters=4, linkage='average')
third_linkage_races = AgglomerativeClustering(n_clusters=3, linkage='ward')

In [ ]:
labels = first_linkage_races.fit_predict(rac_normalized)
races_df['cluster'] = labels

In [ ]:
# plot all pairs of values and color by cluster
sns.pairplot(races_df, hue='cluster')
plt.show()

In [ ]:
avg_cycl, _ = get_average_per_cluster(labels, races_df)
avg_cycl

The first cluster denotes shorter, steeper races that proved to be more challenging, considering the very slow speed achieved albeit being the cluster with the highest startlist_quality. The cluster is relatively small, hinting that races that are this difficult only compose a small subset of all the races. <br>
The second cluster, which is the biggest, with an average length of 200km, denotes all the longer races, which are more endurance-focused. An higher average speed wrt to the first cluster is expected because the steepness dramatically falls, indicating less aggressive albeit longer climbs, while it is also expected to be smaller than the next cluster because of its length. Here, the startlist quality is the smallest, indicating with the fact that the cluster size is so high, that races are more common and faced by more people with different skill levels. <br>
The third cluster indicates races that are on almost flat terrain, with the highest average speeds. These sprint-races present a bit of a higher startlist quality that indicates they are dwelled by slightly more professional individuals, instead of being the 3000 participants marathons.

Since it's the best one in terms of silhouette and looks reasonably well divided, let's visualize it with UMAP

In [ ]:
reducer_races = umap.UMAP(n_neighbors=15, learning_rate=0.1, n_components=3, random_state=RANDOM_STATE)

In [ ]:
reduced_data = reducer_races.fit_transform(rac_normalized)

In [ ]:
# Prepare data for Plotly
df = pd.DataFrame({
    'UMAP 1': reduced_data[:, 0],
    'UMAP 2': reduced_data[:, 1],
    'UMAP 3': reduced_data[:, 2],
    'Cluster': labels
})

# Create interactive 3D scatter plot
fig = px.scatter_3d(
    df.iloc[:5000],
    x='UMAP 1',
    y='UMAP 2',
    z='UMAP 3',
    color='Cluster',
    title='UMAP 3D',
    color_continuous_scale='Viridis'
)

fig.show()

We had to reduce the number of plotted points to keep my laptop from imploding into a singularity.
It looks like there is a central bulk (the long races) from which the first and the second cluster "escape from", to characterize themselves in different ways. Homogeneity looks all but kept, mainly because of the giant differences in size of the clusters and the sample of data to represent.

## Second Linkage

In [ ]:
labels = second_linkage_races.fit_predict(rac_normalized)
races_df['cluster'] = labels

In [ ]:
# plot all pairs of values and color by cluster
sns.pairplot(races_df, hue='cluster')
plt.show()

In [ ]:
avg_cycl, _ = get_average_per_cluster(labels, races_df)
avg_cycl

Same method as before, but now with an additional cluster!
We can see that the split has been made on the last cluster, giving off a shorter, steeper and slower cluster as cluster 1. The mean startlist quality is low, meaning that these kind of races were boosted in rating by the ones coming from cluster 3, which is composed of (probably, considering the startlist quality) more prestigious races

## Third Linkage

In [ ]:
labels = third_linkage_races.fit_predict(rac_normalized)
races_df['cluster'] = labels

In [ ]:
# plot all pairs of values and color by cluster
sns.pairplot(races_df, hue='cluster')
plt.show()

In [ ]:
avg_cycl, _ = get_average_per_cluster(labels, races_df)
avg_cycl

The first blob-cluster stays pretty much the same, with just 41 more datapoints. <br>
The second cluster is, again, very similar to the average method. <br>
The third cluster encapsulates the remaining records plus some of the races lost by the other clusters. <br>
What we can see is that Ward's method gives off clusters with a bit less skewed values of steepness. Nothing that much more interesting